<a href="https://colab.research.google.com/github/epodkwan/growthfunction/blob/main/bsplineemulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 180 kB 13.8 MB/s 
     |████████████████████████████████| 145 kB 73.8 MB/s 
     |████████████████████████████████| 217 kB 41.6 MB/s 
     |████████████████████████████████| 1.0 MB 63.0 MB/s 
     |████████████████████████████████| 51 kB 3.3 MB/s 
     |████████████████████████████████| 76 kB 2.3 MB/s 
  Created wheel for jax: filename=jax-0.3.16-py3-none-any.whl size=1197715 sha256=39eb7bab7005ff0ca2f8dbf4dce8f3b56be82c5b433875e68b9e26f643d5fe38
  Stored in directory: /root/.cache/pip/wheels/b9/0d/a2/2091279bf29b90a2211a271f6f006ef13a656211a9e69ba1e3
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.3.14
    Uninstalling jax-0.3.14:
      Successfully uninstalled jax-0.3.14


In [ ]:
import random
import statistics
from typing import Sequence
import jax
import optax
import numpy as np
import jax.numpy as jnp
from jax import jit,vmap
from flax import linen as nn
from flax.training import train_state,checkpoints
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
class SimpleMLP(nn.Module):
    features:Sequence[int]
    nodes:int

    @nn.compact
    def __call__(self,inputs):
        x=inputs
        for i,feat in enumerate(self.features):
            x=nn.Dense(feat)(x)
            x=jnp.sin(x)
        t=nn.Dense(nodes-1)(x)
        c=nn.Dense(nodes+1)(x)
        t=jnp.concatenate([jnp.zeros((t.shape[0],4)),jnp.cumsum(jax.nn.softmax(t),axis=1),jnp.ones((t.shape[0],3))],axis=1)
        c=jnp.concatenate([jnp.zeros((c.shape[0],1)),c],axis=1)
        return t,c

In [ ]:
def npy_loader(path):
    return jnp.load(path)

In [ ]:
@jit
def _deBoorVectorized(x,t,c):
    p=3
    k=jnp.digitize(x,t)-1
    d=[c[j+k-p] for j in range(0,p+1)]
    for r in range(1,p+1):
        for j in range(p,r-1,-1):
            alpha=(x-t[j+k-p])/(t[j+1+k-r]-t[j+k-p])
            d[j]=(1.0-alpha)*d[j-1]+alpha*d[j]
    return d[p]

In [ ]:
layer_sizes=[64,64]
nodes=16
learning_rate=1e-6
model=SimpleMLP(features=layer_sizes,nodes=nodes)
temp=jnp.array([[1]])
params=model.init(jax.random.PRNGKey(0),temp)
tx=optax.adam(learning_rate=learning_rate)
opt_state=tx.init(params)
deBoor=vmap(_deBoorVectorized,in_axes=(None,0,0))
state=train_state.TrainState.create(apply_fn=model.apply,params=params,tx=tx)

In [ ]:
@jit
def eval_func(params,x,a):
    t,c=restored_state.apply_fn(params,x)
    preds=deBoor(jnp.clip(a,0,0.99999),t,c)
    return preds

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
restored_state=checkpoints.restore_checkpoint(ckpt_dir="/content/drive/My Drive/Colab Notebooks/retrain/checkpoint_0",target=state)
cosmo=npy_loader("/content/drive/My Drive/Colab Notebooks/data1/cosmo.npy")
cosmo_num=random.randrange(1000)
parameters=np.array([[cosmo[cosmo_num,0]]])
input_result=npy_loader("/content/drive/My Drive/Colab Notebooks/data1/combined.npy")
check_result=npy_loader("/content/drive/My Drive/Colab Notebooks/data1check/"+str(cosmo_num)+".npy")
a=npy_loader("/content/drive/My Drive/Colab Notebooks/data1/999.npy")[0,:]
d_data=input_result[cosmo_num,:]
d_test=eval_func(restored_state.params,parameters,a).reshape(-1)
d_check=eval_func(restored_state.params,parameters,check_result[0,:]).reshape(-1)
discrepancy=d_test/d_data
# discrepancy=d_check/check_result[1,:]
fig,(ax1,ax2)=plt.subplots(2,1,constrained_layout=True)
ax1.plot(a,d_data,label="Reference")
ax1.plot(a,d_test,label="Fitting")
ax1.plot(check_result[0,:],d_check,label="Verify")
ax1.set_ylabel("D")
ax1.legend()
ax1.set_title("Omega_m="+str(np.round(parameters[0,0],3)))
ax2.plot(a,discrepancy,label="Discrepancy")
ax2.set_xlabel("a")
ax2.set_ylabel("Predict/Data")
ax2.ticklabel_format(useOffset=False)
ax2.legend()
plt.savefig("/content/drive/My Drive/Colab Notebooks/cosmo"+str(cosmo_num)+".png")
a_plot=[]
med=[]
mean_error=[]
std=[]
for i in range(10):
    plt.clf()
    temp=[]
    for j in range(1000):
        cosmo_num=j
        d_data=input_result[cosmo_num,:]
        parameters[0,0]=cosmo[cosmo_num,0]
        d_test=eval_func(restored_state.params,parameters,a).reshape(-1)
        temp.append((d_test[i*28]/d_data[i*28]-1).item())
        plt.scatter(cosmo[cosmo_num,0].item(),temp[-1],c='b')
    plt.xlabel("Omega_m")
    plt.ylabel("Fractional Error")
    plt.title("Fractional Error of Cosmos (a="+str(np.round(a[i*28].item(),3))+")")
    plt.savefig("/content/drive/My Drive/Colab Notebooks/error"+str(i)+".png")
    a_plot.append(a[i*28].item())
    med.append(statistics.median(temp))
    mean_error.append(statistics.mean(temp))
    std.append(statistics.stdev(temp))
plt.clf()
fig,ax=plt.subplots(constrained_layout=True)
ax.plot(a_plot,med,label="Median")
ax.errorbar(a_plot,mean_error,std,label="Mean")
ax.set_xscale('log')
ax.set_xlabel("a")
ax.set_ylabel("Fractional Error")
ax.set_title("Fractional Error")
ax.legend()
plt.savefig("/content/drive/My Drive/Colab Notebooks/centralerror.png")
drive.flush_and_unmount()